# Tuning CNN hyperparameters with hyperas
This notebook is an example of using hyperas to optimize hyperparameters in a convolutional neural network trained on the mnist data set. <br>
Hyperas: https://github.com/maxpumperla/hyperas

#### Imports

In [ ]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
import tensorflow
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from keras.utils.np_utils import to_categorical

### Data
Define function for providing data

In [ ]:
def data():
    """
    Function that provides data. It will be called only once in the
    optimization process.
    """

    (X_train, y_train), (X_test, y_test) = cifar10.load_data()  

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    X_train /= 255
    X_test /= 255
    
    return X_train, y_train, X_test, y_test

### CNN model
Define model for training CNN. Hyperas will try to minimize the loss

In [ ]:
def cnn(X_train, y_train, X_test, y_test):
    """
    Builds and trains CNN model
    """
    # Define model
    # Parameters to be tuned are number of filters in the two conv layers
    # and the size of the dense layer
    model = Sequential()
    
    model.add(Conv2D({{choice([32, 64])}}, (3,3), activation='relu', input_shape=(32,32,3)))
    model.add(Conv2D({{choice([32, 64])}}, (3,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense({{choice([128, 256, 512])}}))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    # Compile model
    # Parameter to be tuned is learning rate
    sgd = SGD(lr={{choice([0.001, 0.01])}}, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['acc'])
    
    # Train model
    # Parameter to be tuned is batch size
    history = model.fit(X_train, y_train, epochs=15, batch_size={{choice([16, 32, 64])}},
                        validation_split=0.2, verbose=0)
    
    train_accuracy = history.history['acc'][-1]
    val_accuracy = history.history['val_acc'][-1]
    
    # negative validation accuracy is returned, this is the metric to be minimized
    return {'loss' : -val_accuracy,
            'results': {'Train accuracy': train_accuracy,
                        'Validation accuracy': val_accuracy},
            'status': STATUS_OK,
            'model': model}

best_run, best_model = optim.minimize(model=cnn,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=10,
                                      trials=Trials(),
                                      notebook_name='cifar10_hyperperas_example',
                                      verbose=False)

### Result

In [ ]:
# Check parameter indeces of best run
best_run

In [ ]:
# Evaluate best model on test images
X_train, y_train, X_test, y_test = data()
print(best_model.metrics_names)
print(best_model.evaluate(X_test, y_test, verbose=0))

### Notes
- hyperperas is sensitive to comments and docstrings. A docstring with multiple lines in the model function throws a "list index out of range" error. 
- Only a few hyperparameters are tuned in this example. Others to be tuned are choice of optimizer, strides, dropout rate etc.